# Init

In [1]:
import sys
import os
import json
import time
import re

from tqdm import tqdm

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data
import sql

In [2]:
data_dir = os.path.join(cwd, 'exp_comparison')

In [3]:
def extract_questions(fpath: str):
    """
    Extracts Question ID and raw_question from the input file and returns 
    a dictionary where keys are the integer forms of multi-digit Question IDs and values are the raw questions.
    """

    with open(fpath, 'r') as f:
        text = ''.join(f.readlines())

    reg_pattern = re.compile('v\d\.\d.\d')
    version = re.search(reg_pattern, fpath)[0]

    question_dict = {}  # Initialize an empty dictionary to store the results
    
    # Regular expression to match the Question ID and capture its integer part (single or multi-digit)
    question_id_pattern = re.compile(r"Question ID: .*?(\d+)")
    
    # Regular expression to match the raw_question value
    raw_question_pattern = re.compile(r'"raw_question":\s*"(.*?)"')
    
    # Find all Question IDs in the text
    question_ids = question_id_pattern.findall(text)

    # Find all raw_question values in the text
    lst = raw_question_pattern.findall(text)
    raw_questions = []
    for i in range(len(lst)):
        if i%2:
            raw_questions.append(lst[i])
    
    # Iterate over both lists and populate the dictionary
    for question_id, raw_question in zip(question_ids, raw_questions):
        question_dict[int(question_id)] = raw_question  # Convert multi-digit Question ID to integer and add to dictionary
    
    return {version: question_dict}  # Return the final dictionary

def find_cases(stage = 'stage_1', agent_name = 'table_finder'):

    tag = f'{stage}-{agent_name}'
    files = [i for i in sorted(os.listdir(data_dir)) if tag in i]
    # print(os.listdir(data_dir))
    # print(tag, files)
    res = {}

    for fname in files:
        if 'BAD' in fname:
            fpath = os.path.join(data_dir, fname)
            res.update(extract_questions(fpath))

    return res

# Stage-1

In [4]:
stage = 'stage_1'

## Table Finder 

In [5]:
agent_name = 'table_finder'

### Craft Prompt

In [6]:
system_prompt = ""

prompt_dir = os.path.join(cwd, 'prompt')
version = 'v2.7.1'
fname = f'table_finder-stage_1-{version}.md'
prompt_fpath = os.path.join(prompt_dir, fname)

with open(prompt_fpath, 'r') as f:
    prompt_template = ''.join(f.readlines())


def parse_database_and_table(query: str) -> dict:
    """
    Parse the given SQL query to return the database and table names in a dictionary.
    
    Args:
    - query (str): The SQL query to parse.
    
    Returns:
    - dict: A dictionary with 'database' and 'table' keys.
    """

    pattern = r'FROM\s+([a-zA-Z0-9_]+)\.([a-zA-Z0-9_]+)'
    match = re.search(pattern, query, re.IGNORECASE)
    
    if match:
        database = match.group(1)
        table = match.group(2)
        return {'database': database, 'table': table}
    
    return {}


def make_prompt(query: str, ner: dict) -> str:

    """
    ner_res: content from the stage_1
    """

    prompt = prompt_template + query

    # ner_result is None
    if not ner['result']:
        return prompt

    ner_content = {}
    ner_content.update(ner['result'][0])

    sql_res = ner['sql']

    for k, v in sql_res.items():
        for j in v:
            if not j['result']:
                continue

            sql_query = j['query']
            
            # add database and table
            ner_content.update(parse_database_and_table(sql_query))
            if ner_content['table'] == 'SecuMain':
                ner_content['market'] = 'A_Stock'
            elif ner_content['table'] == 'HK_SecuMain':
                ner_content['market'] = 'HK_Stock'
            elif ner_content['table'] == 'US_SecuMain':
                ner_content['market'] = 'US_Stock'
            else:
                pass
            ner_content['data_from_table'] = j['result']

    # add NER result

    ner_str = f"\n\n### **Name Entity Recognition Result**\n```json\n{json.dumps(ner_content, ensure_ascii=False,indent=2)}\n```"

    prompt += ner_str
    
    return prompt

In [7]:
question_path = os.path.join(cwd, 'answer_tmp' + os.sep + 'stage_1-glm_4_plus-ner-v2.0.0-sql-HF-Post.json')

questions = parse_data.read_json(question_path)

### Cases

In [8]:
special_cases = {
    # v2.6.4: deepseek-v3 wrong, glm-4-plus correct
    # first, find the industryID, then find the total value
    # the previous versions all failed
    "72": "2020-07-02风电零部件行业的总市值是多少(元)？",
    # deepseek-v3 always failed to find the correct database
    # AStockOperationsDB => AStockShareholderDB
    # market: US and CN
    "73": "软通动力在2019年报酬总额和领取报酬的管理层人数是多少？",
    # too many related tables
    # deepseek-v3 always correct, glm-4-plus very unstable
    "78": "许继电气在2021年发布了多少条重大事项公告？",
    # unstable correctness: glm-4-plus
    "87": "健康元药业集团股份有限公司在2020-2021年期间进行了几次股份回购？每次回购的金额(单位：万元，保留两位小数)和股数分别是多少？",
    # glm-4-plus: always failed to find the correct database
    # InstitutionDB => AStockEventsDB '上市公司公告资讯/重大事项'
    "94": "华峰化学在2019年发生了哪些舆情事件？请列出事件发生时间(YYYY-MM-DD)、事件名称和情感方向。",
    # new bad cases: need find geo id of CN first
    "41": "2020年成立的CN公司有多少家？",
    # Required a JOIN sql query
    # LC_InvestorDetail has an associated key with LC_InvestorRa
    "100": "横店东磁在2019年1月份进行了多少次投资者关系活动？这些活动中参与机构最多的一次活动发生在哪天(xxxx-xx-xx的格式)，有多少家机构参与？",
    

}

In [10]:
cases = find_cases(stage = 'stage_1', agent_name = 'table_finder')

print(json.dumps(cases['v2.7.1'], ensure_ascii=False, indent=2))

{
  "100": "横店东磁在2019年1月份进行了多少次投资者关系活动？这些活动中参与机构最多的一次活动发生在哪天(xxxx-xx-xx的格式)，有多少家机构参与？",
  "21": "科达制造2021年8月4日当天的最高价与最低价分别是多少",
  "44": "2021年08月哪支基金税后分红最高",
  "58": "2021年1月11日，正常交易且跳空低开的股票一共有几只？",
  "59": "2019下半年，成交量创近一季度新高的证券数量最多的交易日是哪一天？",
  "60": "2021下半年，成交量创近一季度新高的证券数量最多的交易日是哪一天，XXXX年XX月XX日？",
  "62": "博时基金公司成立于（XXXX年XX月XX日）？",
  "73": "软通动力在2019年报酬总额和领取报酬的管理层人数是多少？",
  "75": "截止至中国软件2021年Q4季度，研发投入总额是多少？（调整后的合并报表）",
  "99": "美年健康在2019年发生的股权质押中，质押比例最大的一笔是哪个股东质押给了谁？质押股数和占总股本比例是多少，保留4位小数？"
}


In [11]:
num = 62
num -= 1

print(make_prompt(questions[num]['team'][0]['question'], questions[num]['ner']['stage_1']))

## **Database-Table Schema**

### ConstantDB

|库名中文|库名英文|表英文|表中文|表描述|数据范围|信息来源|
|---|---|---|---|---|---|---|
| 常量库| ConstantDB| HK_SecuMain| 港股证券主表 | 记录港股单个证券品种的简称、中英文名、上市交易、上市状态所等基础信息。 || 
| 常量库 | ConstantDB| US_SecuMain| 美股证券主表 | 记录美国等境外市场单个证券品种的简称、中英文名、上市交易所、上市状态等基础信息。|||
| 常量库 | ConstantDB| SecuMain | 证券主表 | 记录A股单个证券品种（股票、基金、债券）的代码、简称、中英文名、上市交易所、上市板块、上市状态等基础信息。 |||
| 常量库 | ConstantDB| CT_SystemConst | 系统常量表 | 本表收录数据库中各种常量值的具体分类和常量名称描述。 |||
| 常量库 | ConstantDB| LC_AreaCode | 国家城市代码表 | 本表收录世界所有国家层面的数据信息和我国不同层级行政区域的划分信息。 |||
| 常量库 | ConstantDB | QT_TradingDayNew | 交易日表(新) | 本表收录各个市场的交易日信息，包括给定日期是否是交易日，是否周、月、季、年最后一个交易日。 |||

### AStockBasicInfoDB

|库名中文|库名英文|表英文|表中文|表描述|数据范围|信息来源|
|---|---|---|---|---|---|---|
| 上市公司基本资料 | AStockBasicInfoDB | LC_StockArchives | 公司概况 | 收录上市公司的基本情况，包括：联系方式、地址邮编、注册信息、中介机构、行业和产品、公司证券品种及背景资料等内容。 |||
| 上市公司基本资料 | AStockBasicInfoDB | LC_NameChange| 公司名称更改状况 | 收录公司名称历次变更情况，包括：中英文名称、中英文缩写名称、更改日期等内容。 |||
| 上市公司基本资料 | AStockBasicInfoDB | LC_Business| 公司经营范围与行业变更 

## SQL Generator

In [19]:
agent_name = 'sql_generator'

### Craft Prompt

In [20]:
system_prompt = ""

prompt_dir = os.path.join(cwd, 'prompt')
version = 'v2.0.0'
fname = f'sql_generator-stage_1-{version}.md'
prompt_fpath = os.path.join(prompt_dir, fname)

with open(prompt_fpath, 'r') as f:
    prompt_template = ''.join(f.readlines())

def make_prompt(data: dict) -> str:

    prompt = prompt_template

    # 
    table_finder_res = data['table_finder']['stage_1'][0]['data_source'][0]
    # del table_finder_res['question']
    table = table_finder_res['table']
    table_finder_res = json.dumps(table_finder_res, ensure_ascii=False, indent=2)
    reg_p = re.compile('<Database and Table>')
    prompt = re.sub(reg_p, table_finder_res, prompt)

    # 
    table_fname = f'{table}-with_table_name.md'
    table_dir = os.path.join(cwd, 'data' + os.sep + 'table-column')
    table_fpath = os.path.join(table_dir, table_fname)
    with open(table_fpath,'r') as f:
        table_schema = ''.join(f.readlines())
    reg_p = re.compile('<Table-Column Schema>')
    prompt = re.sub(reg_p, table_schema, prompt)

    # 
    if data['ner']['stage_1']['result']:
        ner_res = [i for i in data['ner']['stage_1']['sql'].values() if i][0]
        ner_res = [i['result'] for i in ner_res if i['result']][0][0]
        ner_res = json.dumps(ner_res, ensure_ascii=False, indent=2)
        reg_p = re.compile('<Background Knowledge>')
        prompt = re.sub(reg_p, ner_res, prompt)
    else:
        reg_p = re.compile('<Background Knowledge>')
        prompt = re.sub(reg_p, '', prompt)
        reg_p = re.compile('## Background Knowledge')
        prompt = re.sub(reg_p, '', prompt)

    # replace query
    query = data['team'][0]['question']
    reg_p = re.compile('<Current Query>')
    prompt = re.sub(reg_p, query, prompt)

    return prompt

In [21]:
fname = 'stage_1-glm_4_plus-table_finder-v2.6.4.json'

question_path = os.path.join(cwd, 'answer_tmp' + os.sep + fname)

questions = parse_data.read_json(question_path)

### Cases

In [ ]:
failed = {
    "v1.0.0":
    {
        2: "今天是2021年12月24日，创近半年新高的股票有几只？",
        13: "今天是2020年10月27日，当日收盘价第3高的港股是？(以下都回答简称)",
        63: "最新更新的2019年度报告中，机构持有无限售流通A股数量合计最多的公司简称是？",
        64: "最新更新的2021年度报告中，机构持有无限售流通A股数量合计最多的公司简称是？",
        74: "天弘增利短债C的基金管理人是谁？",
        84: "永泰能源在2020年发生了几次业务范围变更？请列出每次变更的具体日期（xxxx-xx-xx的格式）",
        97: "深科技在2021年12月24日的交易数据如何?具体包括收盘价、成交量、换手率，保留2位小数。"
    },
    "v1.1.0":
    {
        13: "今天是2020年10月27日，当日收盘价第3高的港股是？(以下都回答简称)",
        58: "2021年1月11日，正常交易且跳空低开的股票一共有几只？",
        59: "2019下半年，成交量创近一季度新高的证券数量最多的交易日是哪一天？",
        63: "最新更新的2019年度报告中，机构持有无限售流通A股数量合计最多的公司简称是？",
        64: "最新更新的2021年度报告中，机构持有无限售流通A股数量合计最多的公司简称是？",
        74: "天弘增利短债C的基金管理人是谁？",
        87: "健康元药业集团股份有限公司在2020-2021年期间进行了几次股份回购？每次回购的金额(单位：万元，保留两位小数)和股数分别是多少？",
    }
}

cases = find_cases(stage='stage_1', agent_name='sql_generator')

print(json.dumps(cases, ensure_ascii=False, indent=2))

In [ ]:
num = 21
num -= 1

print(make_prompt(questions[num]))